In [18]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
df_raw= pd.read_csv('../10 data/rotten_tomatoes_movie_reviews.csv')

In [3]:
df = pd.read_csv('../10 data/rotten_tomatoes_movies.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143258 entries, 0 to 143257
Data columns (total 16 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    143258 non-null  object 
 1   title                 142891 non-null  object 
 2   audienceScore         73248 non-null   float64
 3   tomatoMeter           33877 non-null   float64
 4   rating                13991 non-null   object 
 5   ratingContents        13991 non-null   object 
 6   releaseDateTheaters   30773 non-null   object 
 7   releaseDateStreaming  79420 non-null   object 
 8   runtimeMinutes        129431 non-null  float64
 9   genre                 132175 non-null  object 
 10  originalLanguage      129400 non-null  object 
 11  director              139041 non-null  object 
 12  writer                90116 non-null   object 
 13  boxOffice             14743 non-null   object 
 14  distributor           23001 non-null   object 
 15  

## 00 데이터 EDA

In [5]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1444963 entries, 0 to 1444962
Data columns (total 11 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   id              1444963 non-null  object
 1   reviewId        1444963 non-null  int64 
 2   creationDate    1444963 non-null  object
 3   criticName      1444963 non-null  object
 4   isTopCritic     1444963 non-null  bool  
 5   originalScore   1009745 non-null  object
 6   reviewState     1444963 non-null  object
 7   publicatioName  1444963 non-null  object
 8   reviewText      1375738 non-null  object
 9   scoreSentiment  1444963 non-null  object
 10  reviewUrl       1234038 non-null  object
dtypes: bool(1), int64(1), object(9)
memory usage: 111.6+ MB


In [6]:
df_raw.nunique()

id                  69263
reviewId          1432569
creationDate         8510
criticName          15510
isTopCritic             2
originalScore        1729
reviewState             2
publicatioName       2707
reviewText        1359771
scoreSentiment          2
reviewUrl         1138350
dtype: int64

In [7]:
# 중복 제거 및 index 초기화
df_raw.drop_duplicates(inplace=True)
df_raw.reset_index(drop=True, inplace=True)

In [8]:
# 리뷰 개수 90% 인 기준 산출
df_raw['id'].value_counts().quantile(0.9)

np.float64(56.0)

In [9]:
# 리뷰가 56개 이상인 영화(id)의 index 추출
selected_index_1 = df_raw['id'].value_counts()[df_raw['id'].value_counts()>=56].index

In [10]:
selected_index_1

Index(['joker_2019', 'once_upon_a_time_in_hollywood', 'avengers_endgame',
       'captain_marvel', 'a_star_is_born_2018', 'black_panther_2018',
       'star_wars_the_rise_of_skywalker', 'the_batman', 'dune_2021',
       'avengers_infinity_war',
       ...
       'ginger_snaps', '1090089-jack_frost', 'test_pattern', 'claires_camera',
       'the_wound_2017', 'the_mad_womens_ball', 'days_of_heaven',
       'broken_2012', '1104207-panic',
       'jeanne_dielman_23_quai_du_commerce_1080_bruxelles'],
      dtype='object', name='id', length=6963)

In [11]:
# 추출된 Index로 df_raw에서 리뷰 데이터 추출
selected_df = df_raw[df_raw['id'].isin(selected_index_1)]

In [12]:
# index 초기화
selected_df.reset_index(drop=True, inplace= True)

In [13]:
selected_df

,id,reviewId,creationDate,criticName,isTopCritic,originalScore,reviewState,publicatioName,reviewText,scoreSentiment,reviewUrl
0,the_duff,2763233,2021-02-02,Richard Crouse,False,3/5,fresh,Richard Crouse,A school comedy so predictable the screenwrite...,POSITIVE,http://www.richardcrouse.ca/the-duff-3-stars-a...
1,the_duff,2692661,2020-05-27,Andrew Galdi,False,NaN,fresh,Movie Bitches,It was good acting and good writing.,POSITIVE,https://www.youtube.com/watch?v=8KeoWwUtXVQ&li...
2,the_duff,2679627,2020-03-26,Avaryl Halley,False,NaN,fresh,Movie Bitches,"Oh dear, am I the Duff?",POSITIVE,https://www.youtube.com/watch?v=8KeoWwUtXVQ&li...
3,the_duff,2615809,2019-08-15,Udita Jhunjhunwala,True,NaN,fresh,Livemint,While director Ari Sandel's adaptation is devo...,POSITIVE,https://www.livemint.com/Leisure/3wDs58QVt0Mtm...
4,the_duff,2586271,2019-05-14,Olivia Luder,True,3/5,fresh,One Room With A View,Not clever enough to be genre-defining in its ...,POSITIVE,https://oneroomwithaview.com/2015/03/06/duff-r...
...,...,...,...,...,...,...,...,...,...,...,...
925866,thor_love_and_thunder,102706151,2022-07-05,Christie Cronan,False,7/10,fresh,Raising Whasians,Solid but not totally sold&#44; Thor&#58; Ragn...,POSITIVE,https://raisingwhasians.com/thor-love-and-thun...
925867,thor_love_and_thunder,102706150,2022-07-05,Ian Sandwell,False,4/5,fresh,Digital Spy,Thor&#58; Love and Thunder is the most enterta...,POSITIVE,https://www.digitalspy.com/movies/a40496050/th...
925868,thor_love_and_thunder,102706149,2022-07-05,Lauren LaMagna,False,8/10,fresh,Next Best Picture,&quot;Thor&#58; Love and Thunder&quot; is a st...,POSITIVE,https://www.nextbestpicture.com/thor-love-and-...
925869,thor_love_and_thunder,102706148,2022-07-05,Jake Cole,True,1/4,rotten,Slant Magazine,Across Taika Waititi&#8217;s film&#44; a war a...,NEGATIVE,https://www.slantmagazine.com/film/thor-love-a...


In [14]:
# 리뷰를 4개이상한 리뷰어들의 index 추출
selected_index_2 = selected_df['criticName'].value_counts()[selected_df['criticName'].value_counts()>=4].index

In [15]:
selected_index_2

Index(['Frank Swietek', 'Roger Moore', 'Brian Orndorf', 'Jeffrey M. Anderson',
       'Rich Cline', 'Nell Minow', 'Laura Clifford', 'James Berardinelli',
       'Dennis Schwartz', 'David Nusair',
       ...
       'Anne Crump', 'Simon Guerrier', 'Valeriy Kolyadych',
       'Juan José Ontiveros', 'Megan Moffat', 'Mekeisha Madden Toby',
       'Hannah Chambers', 'Philip Horne', 'Jerrica Tisdale', 'Rob Beames'],
      dtype='object', name='criticName', length=6093)

In [16]:
# 추출된 Index로 selected_df에서 리뷰 데이터(final_df) 추출
final_df = selected_df[selected_df['criticName'].isin(selected_index_2)]
final_df

,id,reviewId,creationDate,criticName,isTopCritic,originalScore,reviewState,publicatioName,reviewText,scoreSentiment,reviewUrl
0,the_duff,2763233,2021-02-02,Richard Crouse,False,3/5,fresh,Richard Crouse,A school comedy so predictable the screenwrite...,POSITIVE,http://www.richardcrouse.ca/the-duff-3-stars-a...
1,the_duff,2692661,2020-05-27,Andrew Galdi,False,NaN,fresh,Movie Bitches,It was good acting and good writing.,POSITIVE,https://www.youtube.com/watch?v=8KeoWwUtXVQ&li...
2,the_duff,2679627,2020-03-26,Avaryl Halley,False,NaN,fresh,Movie Bitches,"Oh dear, am I the Duff?",POSITIVE,https://www.youtube.com/watch?v=8KeoWwUtXVQ&li...
3,the_duff,2615809,2019-08-15,Udita Jhunjhunwala,True,NaN,fresh,Livemint,While director Ari Sandel's adaptation is devo...,POSITIVE,https://www.livemint.com/Leisure/3wDs58QVt0Mtm...
4,the_duff,2586271,2019-05-14,Olivia Luder,True,3/5,fresh,One Room With A View,Not clever enough to be genre-defining in its ...,POSITIVE,https://oneroomwithaview.com/2015/03/06/duff-r...
...,...,...,...,...,...,...,...,...,...,...,...
925866,thor_love_and_thunder,102706151,2022-07-05,Christie Cronan,False,7/10,fresh,Raising Whasians,Solid but not totally sold&#44; Thor&#58; Ragn...,POSITIVE,https://raisingwhasians.com/thor-love-and-thun...
925867,thor_love_and_thunder,102706150,2022-07-05,Ian Sandwell,False,4/5,fresh,Digital Spy,Thor&#58; Love and Thunder is the most enterta...,POSITIVE,https://www.digitalspy.com/movies/a40496050/th...
925868,thor_love_and_thunder,102706149,2022-07-05,Lauren LaMagna,False,8/10,fresh,Next Best Picture,&quot;Thor&#58; Love and Thunder&quot; is a st...,POSITIVE,https://www.nextbestpicture.com/thor-love-and-...
925869,thor_love_and_thunder,102706148,2022-07-05,Jake Cole,True,1/4,rotten,Slant Magazine,Across Taika Waititi&#8217;s film&#44; a war a...,NEGATIVE,https://www.slantmagazine.com/film/thor-love-a...


In [32]:
final_df.reset_index(drop=True,inplace=True)

In [37]:
X_train, X_test, Y_train, Y_test = train_test_split(final_df.drop('reviewState', axis=1), final_df['reviewState'], stratify=final_df.criticName, test_size=0.1)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, stratify=X_train.criticName, test_size=0.25)

In [38]:
X_train

,id,reviewId,creationDate,criticName,isTopCritic,originalScore,publicatioName,reviewText,scoreSentiment,reviewUrl
808796,phone_booth,1124119,2003-04-03,Terry Lawson,True,2/4,Detroit Free Press,Despite the film's blessed brevity -- it's onl...,NEGATIVE,http://ae.freep.com/entertainment/ui/michigan/...
754616,sweet_sixteen,1139692,2003-05-16,Jack Mathews,True,3/4,New York Daily News,It's impossible to tell the experienced from t...,POSITIVE,NaN
521045,jem_and_the_holograms_2015,2461936,2018-03-02,Andre Meadows,False,NaN,Black Nerd Comedy,This movie doesn't give you anything different...,NEGATIVE,https://www.youtube.com/watch?v=XfdeyS6Ukg0
651386,boogie_nights,102767671,2023-01-10,Dan DiNicola,False,A+,"The Daily Gazette (Schenectady, NY)",A boldly original and superbly entertaining po...,POSITIVE,https://news.google.com/newspapers?id=KkNGAAAA...
860376,war,1683367,2007-10-25,Joseph Proimakis,False,1.5/5,Movies for the Masses,Proefibikon prodiagrafon astynomiki peripeteia...,NEGATIVE,http://mftm.blogspot.com/2007/10/war-2007.html
...,...,...,...,...,...,...,...,...,...,...
212975,the_girl_who_kicked_the_hornets_nest,1939534,2010-10-29,Edward Douglas,False,7.5/10,ComingSoon.net,'Hornet's Nest' wraps everything up in a nice ...,POSITIVE,http://www.comingsoon.net/news/weekendwarriorn...
314974,yards,170066,2000-01-01,Michael Rechtshaffen,True,NaN,Hollywood Reporter,More preoccupied with themes and moods than pr...,NEGATIVE,NaN
259945,bringing_out_the_dead,165905,2000-01-01,Harvey S. Karten,False,8/10,Compuserve,Nicolas Cage's finest performance since Leavin...,POSITIVE,http://reviews.imdb.com/Reviews/214/21420
426347,getaway,2161626,2013-08-30,Todd Jorgenson,False,NaN,Cinemalogue,The main concern for viewers will be whether t...,NEGATIVE,http://www.cinemalogue.com/2013/08/30/getaway/
